In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net")

In [0]:
df = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/products")

df.display(10)

In [0]:
df_products = (
    spark.readStream 
    .format("cloudFiles").option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("mode", "PERMISSIVE")
    .option("cloudFiles.schemaHints", "product_id string, product_category_name string, product_name_lenght INT, product_description_lenght INT, product_photos_qty INT, product_weight_g INT, product_length_cm INT, product_height_cm INT, product_width_cm INT")
    .option("cloudFiles.schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/products")
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/products/")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/products") 
    .table("bronze.products")
)

In [0]:
%sql
describe bronze.products;

In [0]:
%sql
SELECT * FROM cloud_files_state('abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/products');

In [0]:
%sql
select * from sales_marketing_ws.bronze.products limit 10;